In [1]:
import json
from datasets import load_dataset
import os
from tqdm import tqdm

/home/vipuser/miniconda3/envs/rag/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
num_samples = 100
seed = 42
full_dataset = load_dataset("lmms-lab/OK-VQA", split="val2014")
full_dataset = full_dataset.shuffle(seed=seed)
dummy_subset = full_dataset.select(range(num_samples))
images_ouput_dir = "data/images/val2014"
annotations_dir = os.path.dirname("data/annotations/okvqa_dummy_100.json")

os.makedirs(annotations_dir, exist_ok=True)
os.makedirs(images_ouput_dir, exist_ok=True)

processed_data = []
for item in dummy_subset:
    clean_item = {
        "id": item["question_id"],
        "question": item["question"],
        "image_id": item["question_id"],
        "golden_answers": item["answers"]
    }
    processed_data.append(clean_item)

    image_obj = item["image"]
    image_filename = f"{item['question_id']}.jpg"
    save_path = os.path.join(images_ouput_dir, image_filename)

    if not os.path.exists(save_path):
        try:
            image_obj.save(save_path, 'JPEG')
        except Exception as e:
            tqdm.write(f"Could not save image {save_path}. Error: {e}")


In [4]:
output_filename = "okvqa_dummy_100.json"
with open(output_filename, 'w', encoding='utf-8') as f:
    json.dump(processed_data, f, indent=4, ensure_ascii=False)

In [1]:
import json

def convert_json_to_jsonl(input_file_path, output_file_path):
    """
    Reads a standard JSON file (expected to be a list of objects) and
    writes it to a JSON Lines (.jsonl) file.

    Args:
        input_file_path (str): The path to the input .json file.
        output_file_path (str): The path to the output .jsonl file.
    """
    try:
        with open(input_file_path, 'r', encoding='utf-8') as infile:
            # 1. 一次性读取整个JSON文件，它会被解析成一个Python列表
            data = json.load(infile)

        # 确保输入文件确实是一个列表
        if not isinstance(data, list):
            print(f"Error: The input JSON file '{input_file_path}' does not contain a list of objects.")
            return

        with open(output_file_path, 'w', encoding='utf-8') as outfile:
            # 2. 遍历列表中的每一个对象（字典）
            for entry in data:
                # 3. 将每个对象转换成一个JSON格式的字符串
                json_string = json.dumps(entry, ensure_ascii=False)
                
                # 4. 将这个字符串写入新文件，并在末尾添加一个换行符
                outfile.write(json_string + '\n')
        
        print(f"Successfully converted '{input_file_path}' to '{output_file_path}'.")
        print(f"Total entries written: {len(data)}")

    except FileNotFoundError:
        print(f"Error: The file '{input_file_path}' was not found.")
    except json.JSONDecodeError:
        print(f"Error: The file '{input_file_path}' is not a valid JSON file.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

In [2]:
input_file = "data/okvqa_dummy_100/dev.json"
output_file = "data/okvqa_dummy_100/dev.jsonl"

convert_json_to_jsonl(input_file, output_file)

Successfully converted 'data/okvqa_dummy_100/dev.json' to 'data/okvqa_dummy_100/dev.jsonl'.
Total entries written: 100
